# Omnipose Segmentation from ImageJ Macro converted image directories

This file is meant to aid in omnipose segmentation in a reproducible and streamlined way to help with automated image analysis especially early QC to adjust experimental and imaging parameters as needed to optimize S/N for the experiment. 

#### Import Necessary packages and Functions

In [1]:
# Imports for all chunks
import os
import shutil
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import numpy as np
from cellpose_omni import models, core
import matplotlib as mpl
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from pathlib import Path
import time


c:\Users\MicrobeJ\anaconda3\envs\omnipose_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


import os
import shutil
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader

# Mapping dictionary for renaming channels
channel_map = {'Phase': 'phase', 'eGFP': 'fish', 'DAPI': 'dapi'}

# Root directory
root_dir = 'C:\\Users\\MicrobeJ\\Downloads\\omnipose_multichannel\\sandbox_2'

# Navigate through directories to find OME.TIFF files and rename them
for biorep_dir in os.listdir(root_dir):
    biorep_path = os.path.join(root_dir, biorep_dir)
    if os.path.isdir(biorep_path):
        for date_strain_dir in os.listdir(biorep_path):
            date_strain_path = os.path.join(biorep_path, date_strain_dir)
            if os.path.isdir(date_strain_path):
                for sub_dir in os.listdir(date_strain_path):
                    sub_dir_path = os.path.join(date_strain_path, sub_dir)
                    if os.path.isdir(sub_dir_path):
                        for img_data_dir in os.listdir(sub_dir_path):
                            img_data_path = os.path.join(sub_dir_path, img_data_dir)
                            if os.path.isdir(img_data_path):
                                for file in os.listdir(img_data_path):
                                    if file.endswith('.ome.tiff') or file.endswith('.ome.tif'):
                                        file_path = os.path.join(img_data_path, file)
                                        
                                        # Read the OME.TIFF file to get channel names
                                        reader = OmeTiffReader(file_path)
                                        ome_metadata = reader.ome_metadata
                                        channel_names = [channel.name for channel in ome_metadata.images[0].pixels.channels]
                                        
                                        # Rename folders and files based on channel names
                                        for i, channel_name in enumerate(channel_names):
                                            # Map the original channel name to the new name using the channel_map dictionary
                                            mapped_name = channel_map.get(channel_name, channel_name)
                                            
                                            # Create the old and new folder names based on channel index
                                            old_folder_name = f"C{i+1}-MAX_sequence"
                                            new_folder_name = f"{mapped_name}-MAX_sequence"
                                            
                                            # Create the full path to the old and new folder names
                                            old_folder_path = os.path.join(img_data_path, old_folder_name)
                                            new_folder_path = os.path.join(img_data_path, new_folder_name)
                                            
                                            # If the old folder exists, rename it to the new folder name
                                            if os.path.exists(old_folder_path):
                                                shutil.move(old_folder_path, new_folder_path)
                                            
                                            # Rename individual single-page TIFF files inside the new folder
                                            for single_tiff in os.listdir(new_folder_path):
                                                # Check if the file starts with the old channel name
                                                if single_tiff.startswith(f"C{i+1}-MAX"):
                                                    # Create the full path to the old single-page TIFF file
                                                    old_single_tiff_path = os.path.join(new_folder_path, single_tiff)
                                                    
                                                    # Create the new single-page TIFF file name based on mapped channel name
                                                    new_single_tiff_name = single_tiff.replace(f"C{i+1}-MAX", f"{mapped_name}-MAX")
                                                    
                                                    # Create the full path to the new single-page TIFF file
                                                    new_single_tiff_path = os.path.join(new_folder_path, new_single_tiff_name)
                                                    
                                                    # Rename the old single-page TIFF file to the new name
                                                    shutil.move(old_single_tiff_path, new_single_tiff_path)
                                            
                                            # Create old and new multi-page TIFF file names based on channel index
                                            old_file_name = f"C{i+1}-MAX.tif"
                                            new_file_name = f"{mapped_name}-MAX.tif"
                                            
                                            # Create the full path to the old and new multi-page TIFF files
                                            old_file_path = os.path.join(img_data_path, old_file_name)
                                            new_file_path = os.path.join(img_data_path, new_file_name)
                                            
                                            # If the old multi-page TIFF file exists, rename it to the new name
                                            if os.path.exists(old_file_path):
                                                shutil.move(old_file_path, new_file_path)




## Running Omnipose for Segmentation

Here is the incorporation into the omnipose script

### Importing the images and QC to check images match expectations

In [ ]:
import numpy as np
from cellpose_omni import models, core, io, transforms
from omnipose.utils import normalize99
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread, imsave

# Check GPU setup
use_GPU = core.use_gpu()
print(f'>>> GPU activated? {use_GPU}')

# Image loading and initial analysis
imgs = [io.imread(f) for f in phase_max_sequence_files]

# Counter for total images
total_images = len(imgs)

# Counter for images with issues
issues_counter = 0

# Quality control and Data check
for idx, i in enumerate(imgs):
    shape = i.shape
    dtype = i.dtype
    min_val, max_val = i.min(), i.max()

    # Add quality control checks here. For example:
    if shape != (512, 512) or min_val < 4000 or max_val > 28000:
        issues_counter += 1
        print(f"Warning: Image at {phase_max_sequence_files[idx]} has issues.")
        print(f"  - Original image shape: {shape}")
        print(f"  - Data type: {dtype}")
        print(f"  - Data range: min {min_val}, max {max_val}")

print(f"\nTotal number of images processed: {total_images}")
if issues_counter:
    print(f"Number of images with issues: {issues_counter}")
else:
    print("No issues found in images.")

### Image Segmetnation and Saving Results


In [ ]:
import time
from cellpose_omni import utils

# Initialize model and parameters
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=False, model_type=model_name)
params = {'channels': [0, 0], 'mask_threshold': -1, 'flow_threshold': 0,
          'niter': None, 'omni': True, 'cluster': True, 'resample': True,
          'verbose': 0, 'affinity_seg': 0}

# Run segmentation
tic = time.time()
masks, flows, styles = model.eval(imgs, **params)
net_time = time.time() - tic
print(f'Total segmentation time: {net_time}s')

# Save results
for idx, mask in enumerate(masks):